In [1]:
# %pip install python-dotenv
# %pip install seaborn
# %pip install tensorflow_data_validation

In [2]:
import os
import pandas as pd
import geopandas as gpd
import pygeos as pg
import numpy as np
# import tensorflow as tf
# import tensorflow_data_validation as tfdv
import sklearn as sk
import scipy as sp
import seaborn as sns
# from datetime import datetime
# from dotenv import load_dotenv
from IPython.display import clear_output
from matplotlib import pyplot as plt
from shapely import wkt

2022-12-15 01:57:00.275052: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-15 01:57:00.400141: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# The following lines adjust the granularity of reporting.
#pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
os.chdir('/tf')

In [6]:
def getGPDfromPD(df: pd.DataFrame, geomCol: str, crs: str = "EPSG:3978") -> gpd.GeoDataFrame:
    """
    Convert a pandas dataframe to a geopandas dataframe
    :param df: pandas dataframe
    :param geomCol: name of the geometry column
    :param crs: coordinate reference system
    :return: geopandas dataframe
    """
    if 'geom' in df.columns:
        df.rename(columns={'geom': 'geometry'}, inplace=True)

    df[geomCol] = df[geomCol].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, geometry=geomCol, crs=crs)
    return gdf

In [ ]:
fireWeatherTable = "Data/FinalFeature.csv"
dfFireWeather = pd.read_csv(fireWeatherTable)

In [ ]:
dfEval = dfFireWeather

In [ ]:
# Store our random selection, run once
# randomTrain = "RandomTrain"
# dfTrain.to_sql(randomTrain, db_push_con, if_exists='replace', index=False)

# randomTest = "RandomTest"
# dfTest.to_sql(randomTest, db_push_con, if_exists='replace', index=False)

# randomValidate = "RandomValidate"
# dfValidate.to_sql(randomValidate, db_push_con, if_exists='replace', index=False)

In [ ]:
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn import svm
# from sklearn.inspection import DecisionBoundaryDisplay


In [ ]:
dfTrim = dfEval.copy(deep=True)

#edit this
dfTrim = dfTrim.drop()

In [ ]:
# randomly select 6 years from 2010-2019 for training
dfTrain = dfTrim[dfTrim['YEAR'].isin([2010, 2011, 2012, 2013, 2014, 2015, 2016])]
dfTest = dfTrim[dfTrim['YEAR'].isin([2017, 2018, 2019, 2020])]

In [ ]:
def scaleData(df: pd.DataFrame) -> pd.DataFrame:
    # regularize y values using z score
    df = (df - df.mean()) / df.std()
    # set max value to 3 zscore
    df[df > 3] = 3
    # set min value to -3 zscore
    df[df < -3] = -3

    # shift the wole train set to be positive
    df = df + 3

    # return dataframe list of means and stds
    return df, df.mean(), df.std()

In [ ]:
dfTrainScaled, dfMeans, dfStdevs = scaleData(dfTrain.copy(deep=True))
dfTrainScaled['YEAR'] = dfTrain['YEAR']

In [ ]:

print(dfTrainScaled.count())
print(dfTrainScaled.isna().sum().sum())
print(dfTrainScaled.dtypes)

In [ ]:
print(dfTrainScaled.describe())

In [ ]:
C = 1  # SVM regularization parameter

model1 = svm.SVC(kernel="linear", C=C, decision_function_shape='ovo'),
model2 = svm.LinearSVC(C=C, max_iter=10000),
model3 = svm.SVC(kernel="rbf", gamma=0.7, C=C, decision_function_shape='ovo'),
model4 = svm.SVC(kernel="poly", degree=3, gamma="auto", C=C, decision_function_shape='ovo'),

dfFeatures = dfTrainScaled.drop(['size_ha_bin', 'YEAR'], axis=1)
dfLabel = dfTrainScaled['size_ha_bin']

model1 = model1.fit(dfFeatures, dfLabel)
model2 = model2.fit(dfFeatures, dfLabel)
model3 = model3.fit(dfFeatures, dfLabel)
model4 = model4.fit(dfFeatures, dfLabel)

